## Valhalla OD-TTM and DTM

In [1]:
import requests
import json

In [2]:


# Valhalla API Endpoint
base_url = "http://localhost:8002"  # Replace with your Valhalla instance URL
matrix_endpoint = "/route"

# Define your locations (MSOA centroids or other points)
locations = [
    [51.5074, -0.1278],  # Example: London coordinates
    [51.4545, -0.9784],  # Example: Another location
    # ... add more locations
]

# Query parameters
params = {
    "json": json.dumps({
        "locations": locations,
        "costing": "auto",  # Adjust for different transport modes (auto, bicycle, pedestrian)
        "directions_options": {"units": "miles"},  # Change units if needed (kilometers, etc.)
        "sources": list(range(len(locations))),
        "targets": list(range(len(locations)))
    })
}

# Send the request
response = requests.get(base_url + matrix_endpoint, params=params)

# Handle the response
if response.status_code == 200:
    data = response.json()
    travel_times = data['sources_to_targets']

    # Process the travel time matrix
    for i, row in enumerate(travel_times):
        for j, time in enumerate(row):
            print(f"Travel time from {i} to {j}: {time} seconds")
else:
    print(f"Error: {response.status_code} - {response.text}")


Error: 400 - {"error_code":100,"error":"Failed to parse json request","status_code":400,"status":"Bad Request"}


In [10]:
url = "http://localhost:8002/route"
headers = {"Content-Type": "application/json"}
data = {
    "locations": [
        {"lat": 51.52352798278855, "lon": -0.13659632037185904},
        {"lat": 51.5289746153648, "lon": -0.1183138174758685}
    ],
    "costing": "auto",
    "directions_options": {"units": "miles"}
}

response = requests.post(url, headers=headers, json=data)
print(response.json())

{'trip': {'locations': [{'type': 'break', 'lat': 51.523527, 'lon': -0.136596, 'original_index': 0}, {'type': 'break', 'lat': 51.528974, 'lon': -0.118313, 'side_of_street': 'right', 'original_index': 1}], 'legs': [{'maneuvers': [{'type': 1, 'instruction': 'Drive southeast.', 'verbal_succinct_transition_instruction': 'Drive southeast. Then Turn left onto University Street.', 'verbal_pre_transition_instruction': 'Drive southeast. Then Turn left onto University Street.', 'verbal_post_transition_instruction': 'Continue for 300 feet.', 'time': 46.952, 'length': 0.058, 'cost': 385.766, 'begin_shape_index': 0, 'end_shape_index': 7, 'verbal_multi_cue': True, 'travel_mode': 'drive', 'travel_type': 'car'}, {'type': 15, 'instruction': 'Turn left onto University Street.', 'verbal_transition_alert_instruction': 'Turn left onto University Street.', 'verbal_succinct_transition_instruction': 'Turn left.', 'verbal_pre_transition_instruction': 'Turn left onto University Street.', 'verbal_post_transition_

In [4]:
import pandas as pd
import datetime

In [6]:
r5_diff = pd.read_csv(r'/mnt/d/University College London/Chen, Huanfa - Dissertation_Mengyu_Ding/Data/greater_london_od_mat_all/re_diff_r5.csv', encoding = 'latin1')

In [4]:
r5_diff.head()

,destinationID,originID,re_diff_tt,origin_lat,origin_lon,destination_lat,destination_lon,euclidean_distance,origin_region,destination_region
0,E02000001,E02000002,-15.267176,51.588242,0.139471,51.514382,-0.092411,0.243361,Outer London,Inner London
1,E02000001,E02000003,-23.106796,51.574923,0.140919,51.514382,-0.092411,0.241056,Outer London,Inner London
2,E02000001,E02000004,-14.425428,51.555446,0.176841,51.514382,-0.092411,0.272366,Outer London,Inner London
3,E02000001,E02000005,-18.777293,51.561414,0.143309,51.514382,-0.092411,0.240367,Outer London,Inner London
4,E02000001,E02000007,-19.117647,51.558095,0.157363,51.514382,-0.092411,0.253570,Outer London,Inner London


In [7]:
# make a copy keep only columns:destinationID	originID origin_lat	origin_lon	destination_lat	destination_lon
valhalla_df = r5_diff.copy()
valhalla_df = valhalla_df[['destinationID','originID','origin_lat','origin_lon','destination_lat','destination_lon']]

In [9]:
valhalla_df.head()

,destinationID,originID,origin_lat,origin_lon,destination_lat,destination_lon,travel_time,travel_distance,other_routing_info
0,E02000001,E02000002,51.588242,0.139471,51.514382,-0.092411,inf,inf,{}
1,E02000001,E02000003,51.574923,0.140919,51.514382,-0.092411,inf,inf,{}
2,E02000001,E02000004,51.555446,0.176841,51.514382,-0.092411,inf,inf,{}
3,E02000001,E02000005,51.561414,0.143309,51.514382,-0.092411,inf,inf,{}
4,E02000001,E02000007,51.558095,0.157363,51.514382,-0.092411,inf,inf,{}


In [8]:
valhalla_url = "http://localhost:8002/route"

departure_time = "2024-08-06T08:30:00"

# Function to get routing information
def get_routing_info(origin, destination):
    request_body = {
        "locations": [
            {"lat": origin[0], "lon": origin[1]},
            {"lat": destination[0], "lon": destination[1]}
        ],
        "costing": "auto",
        "directions_options": {"units": "miles"}
    }
    response = requests.post(valhalla_url, json=request_body)
    data = response.json()
    
    if "trip" in data and "summary" in data["trip"]:
        travel_time = data["trip"]["summary"]["time"]
        travel_distance = data["trip"]["summary"]["length"]
        other_routing_info = data["trip"]["summary"]
        return travel_time, travel_distance, other_routing_info
    else:
        return float('inf'), float('inf'), {}

# Add new columns to the dataframe
valhalla_df['travel_time'] = None
valhalla_df['travel_distance'] = None
valhalla_df['other_routing_info'] = None


# Function to get routing information with departure time
def get_routing_info_with_departure(origin, destination, departure_time):
    request_body = {
        "locations": [
            {"lat": origin[0], "lon": origin[1]},
            {"lat": destination[0], "lon": destination[1]}
        ],
        "costing": "auto",
        "date_time": {
            "type": 1,  # 0 for current, 1 for departure, 2 for arrival
            "value": departure_time
        },
        #"directions_options": {"units": "miles"}
        "directions_options": {"units": "kilometers"}
    }
    response = requests.post(valhalla_url, json=request_body)
    data = response.json()
    
    if "trip" in data and "summary" in data["trip"]:
        travel_time = data["trip"]["summary"]["time"]
        travel_distance = data["trip"]["summary"]["length"]
        other_routing_info = data["trip"]["summary"]
        return travel_time, travel_distance, other_routing_info
    else:
        return float('inf'), float('inf'), {}


# print date time


start_run_time = datetime.datetime.now()
print("start time:")
print(start_run_time)


# Iterate through the dataframe and populate the new columns
counter = 0
for index, row in valhalla_df.iterrows():

    origin = (row['origin_lat'], row['origin_lon'])
    destination = (row['destination_lat'], row['destination_lon'])
    
    #travel_time, travel_distance, other_routing_info = get_routing_info_with_departure(origin, destination, departure_time)
    travel_time, travel_distance, other_routing_info = get_routing_info(origin, destination)
    valhalla_df.at[index, 'travel_time'] = travel_time
    valhalla_df.at[index, 'travel_distance'] = travel_distance
    valhalla_df.at[index, 'other_routing_info'] = other_routing_info
    # 
    counter += 1
    #
    print(f"Processed {counter} pairs...")    

# Convert other_routing_info to string for better readability
valhalla_df['other_routing_info'] = valhalla_df['other_routing_info'].apply(lambda x: str(x))


end_run_time = datetime.datetime.now()
print("end time:")
print(end_run_time)

# Calculate the total run time
run_time = end_run_time - start_run_time

print(f"Total run time: {run_time}")

# Display the updated dataframe
print(valhalla_df.head())

start time:
2024-08-06 11:36:24.358860
Processed 1 pairs...
Processed 2 pairs...
Processed 3 pairs...
Processed 4 pairs...
Processed 5 pairs...
Processed 6 pairs...
Processed 7 pairs...
Processed 8 pairs...
Processed 9 pairs...
Processed 10 pairs...
Processed 11 pairs...
Processed 12 pairs...
Processed 13 pairs...
Processed 14 pairs...
Processed 15 pairs...
Processed 16 pairs...
Processed 17 pairs...
Processed 18 pairs...
Processed 19 pairs...
Processed 20 pairs...
Processed 21 pairs...
Processed 22 pairs...
Processed 23 pairs...
Processed 24 pairs...
Processed 25 pairs...
Processed 26 pairs...
Processed 27 pairs...
Processed 28 pairs...
Processed 29 pairs...
Processed 30 pairs...
Processed 31 pairs...
Processed 32 pairs...
Processed 33 pairs...
Processed 34 pairs...
Processed 35 pairs...
Processed 36 pairs...
Processed 37 pairs...
Processed 38 pairs...
Processed 39 pairs...
Processed 40 pairs...
Processed 41 pairs...
Processed 42 pairs...
Processed 43 pairs...
Processed 44 pairs...
Pr

KeyboardInterrupt: 

In [15]:
valhalla_df.to_csv(r'/mnt/d/University College London/Chen, Huanfa - Dissertation_Mengyu_Ding/Data/valhalla_df_raw.csv', index=False)
#Total run time: 1 day, 1:49:55.114575

# try ttm mode:

In [15]:
# print system time:
import datetime
print(datetime.datetime.now())

2024-08-06 11:44:29.832896


In [13]:
from freezegun import freeze_time

In [17]:
with freeze_time("2024-07-23 08:30:00"):
    print("Current time:", datetime.datetime.now())

Current time: 2024-07-23 08:30:00


In [16]:
print(datetime.datetime.now())

2024-08-06 11:44:41.323799


In [18]:
valhalla_url = "http://localhost:8002/route"

departure_time = "2024-08-06T08:30:00"
with freeze_time("2024-07-23 08:30:00"):
    # Function to get routing information
    def get_routing_info(origin, destination):
        request_body = {
            "locations": [
                {"lat": origin[0], "lon": origin[1]},
                {"lat": destination[0], "lon": destination[1]}
            ],
            "costing": "auto",
            "directions_options": {"units": "miles"}
        }
        response = requests.post(valhalla_url, json=request_body)
        data = response.json()
        
        if "trip" in data and "summary" in data["trip"]:
            travel_time = data["trip"]["summary"]["time"]
            travel_distance = data["trip"]["summary"]["length"]
            other_routing_info = data["trip"]["summary"]
            return travel_time, travel_distance, other_routing_info
        else:
            return float('inf'), float('inf'), {}

    # Add new columns to the dataframe
    valhalla_df['travel_time'] = None
    valhalla_df['travel_distance'] = None
    valhalla_df['other_routing_info'] = None


    # Function to get routing information with departure time
    def get_routing_info_with_departure(origin, destination, departure_time):
        request_body = {
            "locations": [
                {"lat": origin[0], "lon": origin[1]},
                {"lat": destination[0], "lon": destination[1]}
            ],
            "costing": "auto",
            "date_time": {
                "type": 1,  # 0 for current, 1 for departure, 2 for arrival
                "value": departure_time
            },
            #"directions_options": {"units": "miles"}
            "directions_options": {"units": "kilometers"}
        }
        response = requests.post(valhalla_url, json=request_body)
        data = response.json()
        
        if "trip" in data and "summary" in data["trip"]:
            travel_time = data["trip"]["summary"]["time"]
            travel_distance = data["trip"]["summary"]["length"]
            other_routing_info = data["trip"]["summary"]
            return travel_time, travel_distance, other_routing_info
        else:
            return float('inf'), float('inf'), {}


    # print date time


    start_run_time = datetime.datetime.now()
    print("start time:")
    print(start_run_time)


    # Iterate through the dataframe and populate the new columns
    counter = 0
    for index, row in valhalla_df.iterrows():

        origin = (row['origin_lat'], row['origin_lon'])
        destination = (row['destination_lat'], row['destination_lon'])
        
        #travel_time, travel_distance, other_routing_info = get_routing_info_with_departure(origin, destination, departure_time)
        travel_time, travel_distance, other_routing_info = get_routing_info(origin, destination)
        valhalla_df.at[index, 'travel_time'] = travel_time
        valhalla_df.at[index, 'travel_distance'] = travel_distance
        valhalla_df.at[index, 'other_routing_info'] = other_routing_info
        # 
        counter += 1
        #
        print(f"Processed {counter} pairs...")    

    # Convert other_routing_info to string for better readability
    valhalla_df['other_routing_info'] = valhalla_df['other_routing_info'].apply(lambda x: str(x))


    end_run_time = datetime.datetime.now()
    print("end time:")
    print(end_run_time)

    # Calculate the total run time
    run_time = end_run_time - start_run_time

    print(f"Total run time: {run_time}")

    # Display the updated dataframe
    print(valhalla_df.head())

start time:
2024-07-23 08:30:00
Processed 1 pairs...
Processed 2 pairs...
Processed 3 pairs...
Processed 4 pairs...
Processed 5 pairs...
Processed 6 pairs...
Processed 7 pairs...
Processed 8 pairs...
Processed 9 pairs...
Processed 10 pairs...
Processed 11 pairs...
Processed 12 pairs...
Processed 13 pairs...
Processed 14 pairs...
Processed 15 pairs...
Processed 16 pairs...
Processed 17 pairs...
Processed 18 pairs...
Processed 19 pairs...
Processed 20 pairs...
Processed 21 pairs...
Processed 22 pairs...
Processed 23 pairs...
Processed 24 pairs...
Processed 25 pairs...
Processed 26 pairs...
Processed 27 pairs...
Processed 28 pairs...
Processed 29 pairs...
Processed 30 pairs...
Processed 31 pairs...
Processed 32 pairs...
Processed 33 pairs...
Processed 34 pairs...
Processed 35 pairs...
Processed 36 pairs...
Processed 37 pairs...
Processed 38 pairs...
Processed 39 pairs...
Processed 40 pairs...
Processed 41 pairs...
Processed 42 pairs...
Processed 43 pairs...
Processed 44 pairs...
Processed

KeyboardInterrupt: 

In [19]:
valhalla_df.head()

,destinationID,originID,origin_lat,origin_lon,destination_lat,destination_lon,travel_time,travel_distance,other_routing_info
0,E02000001,E02000002,51.588242,0.139471,51.514382,-0.092411,1745.798,13.352,"{'has_time_restrictions': False, 'has_toll': T..."
1,E02000001,E02000003,51.574923,0.140919,51.514382,-0.092411,1693.07,13.537,"{'has_time_restrictions': False, 'has_toll': T..."
2,E02000001,E02000004,51.555446,0.176841,51.514382,-0.092411,1794.809,14.926,"{'has_time_restrictions': True, 'has_toll': Fa..."
3,E02000001,E02000005,51.561414,0.143309,51.514382,-0.092411,1674.209,13.256,"{'has_time_restrictions': True, 'has_toll': Fa..."
4,E02000001,E02000007,51.558095,0.157363,51.514382,-0.092411,1749.214,13.672,"{'has_time_restrictions': True, 'has_toll': Fa..."


In [ ]:
valhalla_df.to_csv(r'/mnt/d/University College London/Chen, Huanfa - Dissertation_Mengyu_Ding/Data/valhalla_df_raw_ttm.csv', index=False)
#Total run time: 1 day, 1:49:55.114575